> Time Series Trial

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Load the dataset
file_path = ''  # Replace with your file path
data = pd.read_excel(file_path)

# Preprocessing data (adjust according to your dataset's structure)
data = data.sort_values('Year')
train_data = data[data['Year'] <= 2018]
test_data = data[data['Year'] >= 2019]

# Scaling the CRK data for the LSTM model
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data['CRK'].values.reshape(-1,1))

# Preparing the training data for LSTM
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 1
X_train, y_train = create_dataset(train_scaled, time_step)

# Reshaping input to be [samples, time steps, features] for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Building the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Training the model
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)

# Forecasting for 2019-2023
input_seq = train_scaled[-time_step:].reshape(1, -1)
input_seq = input_seq.reshape((1, time_step, 1))
predicted = []
for i in range(len(test_data)):
    # Making prediction
    yhat = model.predict(input_seq, verbose=0)
    predicted.append(yhat[0,0])
    # Updating the input sequence
    input_seq = np.append(input_seq[:,1:,:], [[yhat]], axis=1)

# Inverse transform to get actual CRK values
predicted = scaler.inverse_transform(np.array(predicted).reshape(-1, 1)).flatten()

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(data['Year'], data['CRK'], label='Actual CRK Ratings', color='blue')
plt.plot(test_data['Year'], predicted, label='Forecasted CRK Ratings', color='red', linestyle='dashed')
plt.xlabel('Year')
plt.ylabel('CRK Rating')
plt.title('LSTM Forecast vs Actual CRK Ratings')
plt.legend()
plt.grid(True)
plt.show()